# Суммаризатор видео

## 0. Устанавливаем необходимые зависимости

In [2]:
!pip install yt-dlp moviepy ffmpeg-python transformers[sentencepiece] faster-whisper torch

## 1. Скачиваем видео по ссылке

In [3]:
from pathlib import Path
import yt_dlp as ytdl

def download_video(url: str, out_dir: str = "downloads") -> Path:
    out_dir_p = Path(out_dir)
    out_dir_p.mkdir(parents=True, exist_ok=True)
    ydl_opts = {
        'outtmpl': str(out_dir_p / '%(title)s.%(ext)s'),
        'format': 'bestvideo+bestaudio/best',
        'noplaylist': True,
        'quiet': True,
        'no_warnings': True,
    }
    with ytdl.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        # инфа содержит 'title' и 'ext'
        filename = ydl.prepare_filename(info)
    return Path(filename)

## 2. Извлечение аудио

## 3. Траскрибация аудио

In [4]:
# Пропускаем скачивание видео, извлечение аудио и транскрибацию
# Используем готовый тестовый текст для суммаризации

text = """
В этом видео рассказывается о красоте математики. Автор объясняет, что математика - это не просто сухие цифры и формулы, 
а удивительный язык, описывающий вселенную. Рассматривается пример числа Пи - бесконечной десятичной дроби, 
которая встречается в самых неожиданных местах: от геометрии круга до квантовой физики. 
Особое внимание уделяется формуле Эйлера, которая элегантно связывает пять фундаментальных математических констант. 
Математика учит логическому мышлению, помогает находить закономерности и решать сложные проблемы. 
Каждая доказанная теорема - это маленькое открытие, приближающее нас к пониманию мира. 
Автор призывает не бояться математики, подчеркивая ее красоту и увлекательность.
Математические концепции могут казаться сложными на первый взгляд, но при глубоком изучении 
открывают свою внутреннюю гармонию и логическую завершенность. Именно эта гармония 
и делает математику по-настоящему красивой наукой, которая помогает нам понимать 
законы природы и создавать новые технологии.
"""

print("ТЕСТОВЫЙ ТРАНСКРИБИРОВАННЫЙ ТЕКСТ:")
print("=" * 50)
print(text)
print("=" * 50)
print(f"Длина текста: {len(text)} символов")

ТЕСТОВЫЙ ТРАНСКРИБИРОВАННЫЙ ТЕКСТ:

В этом видео рассказывается о красоте математики. Автор объясняет, что математика - это не просто сухие цифры и формулы, 
а удивительный язык, описывающий вселенную. Рассматривается пример числа Пи - бесконечной десятичной дроби, 
которая встречается в самых неожиданных местах: от геометрии круга до квантовой физики. 
Особое внимание уделяется формуле Эйлера, которая элегантно связывает пять фундаментальных математических констант. 
Математика учит логическому мышлению, помогает находить закономерности и решать сложные проблемы. 
Каждая доказанная теорема - это маленькое открытие, приближающее нас к пониманию мира. 
Автор призывает не бояться математики, подчеркивая ее красоту и увлекательность.
Математические концепции могут казаться сложными на первый взгляд, но при глубоком изучении 
открывают свою внутреннюю гармонию и логическую завершенность. Именно эта гармония 
и делает математику по-настоящему красивой наукой, которая помогает нам понимать 


## 4. Суммвризация текста

In [5]:
def chunk_text(text: str, max_chars: int = 3000):
    chunks = []
    start = 0
    while start < len(text):
        end = start + max_chars
        chunks.append(text[start:end])
        start = end
    return chunks

In [6]:
from transformers import pipeline

def summarize_text(text: str, model_name: str, max_chunk_chars: int = 3000) -> str:
    summarizer = pipeline('summarization', model=model_name, truncation=True)
    chunks = chunk_text(text, max_chars=max_chunk_chars)
    summaries = []
    for i, ch in enumerate(chunks):
        res = summarizer(ch, max_length=200, min_length=30, do_sample=False)
        summaries.append(res[0]['summary_text'])

    if len(summaries) > 1:
        joined = ' '.join(summaries)
        res2 = summarizer(joined, max_length=250, min_length=50, do_sample=False)
        return res2[0]['summary_text']
    else:
        return summaries[0]

In [7]:
summarize_text(text, 'sshleifer/distilbart-cnn-12-6')

Device set to use cpu


' \xa0\xa0- \xa0-\xa0\xa0\xa0‘‘Математика’ - \xa0–\xa0- ‘.’ – \xa0‚аганная теоре’, ‘.-‘-“” –\xa0“\xa0” - “\xa0-” ‘- “-\u2009”:\xa0\u2009гера.гифра, \u2009Љ’: “.-” \xa0"- \u2009\u2009\xa0’-  \u2009-  \xa0Љ-’s ‘\u2009- ”- ’'

In [8]:
import torch
from transformers import GPT2Tokenizer, T5ForConditionalGeneration 
tokenizer = GPT2Tokenizer.from_pretrained('RussianNLP/FRED-T5-Summarizer',eos_token='</s>')
model = T5ForConditionalGeneration.from_pretrained('RussianNLP/FRED-T5-Summarizer')
device='cpu'
model.to(device)

#input_text='<LM> Сократи текст.\n В деревне, затерянной среди зелёных холмов, жил старик по имени Иван. Его жизнь протекала медленно и размеренно. Каждое утро Иван выходил на поля, чтобы заботиться о своём скромном участке земли. Он выращивал картофель и морковь, которые были его главным источником пищи. Вечера старик проводил у камина, читая книги и вспоминая молодость. Жизнь в деревне была тяжёлая, но Иван находил в ней простые радости.'
input_ids=torch.tensor([tokenizer.encode(text)]).to(device)
outputs=model.generate(input_ids,eos_token_id=tokenizer.eos_token_id,
                    num_beams=5,
                    min_new_tokens=17,
                    max_new_tokens=200,
                    do_sample=True,
                    no_repeat_ngram_size=4,
                    top_p=0.9)
print(tokenizer.decode(outputs[0][1:]))

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  6.31it/s]


В этом видео рассказывается о красоте математики. Автор объясняет, что математика - это не просто сухие цифры и формулы, 
а удивительный язык, описывающий вселенную. Рассматривается пример числа Пи - бесконечной десятичной дроби, 
которая встречается в самых неожиданных местах: от геометрии круга до квантовой физики. Особое внимание уделяется формуле Эйлера, которая элегантно связывает пять фундаментальных математических констант. Автор призывает не бояться математики, подчеркивая ее красоту и увлекательность.</s>


In [9]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("cointegrated/rut5-base-absum")
model = T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-base-absum")

inputs = tokenizer(text, return_tensors='pt')
with torch.no_grad():
    hypotheses = model.generate(
        **inputs, 
        do_sample=True, top_p=0.95, num_return_sequences=10, 
        repetition_penalty=2.5,
        max_length=32,
    )
for h in hypotheses:
    print(tokenizer.decode(h, skip_special_tokens=True))

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Огромное удовольствие — в этой видео приводит сюда мастер-класс «Декорийные триугольники»
В этом видео выставка немного посвящена вопросам уроков. На телевидении представление отряда из научных взглядов «
Смотрите видео, рассказывающие об красоте математики. Программа наносит впечатление про любую науку
На видео обзора показаны несколько способов выяснить красоту математики.
В этом видео рассказывается, какие имеет свой внутренний язык. На следующий день он показывает картину классификации «М
Какую глубокое изучение изучают математику?
В этом видео рассказывается о красоте математики.
Затем и покажу подробное видео в поддержку исторической науке, создавающей новую технологию.
Итого же можно выяснить в продемонстрированном видео.
С ученым издания проходит тематическое видео, посвященное теме математики. В конце автор рассказывает свою удивительную поэтике


In [10]:
# НОВАЯ МОДЕЛЬ 1: IlyaGusev/rut5_base_sum_gazeta
print("=== НОВАЯ МОДЕЛЬ 1: IlyaGusev/rut5_base_sum_gazeta ===")
print("Загрузка модели...")

try:
    from transformers import pipeline
    
    summarizer = pipeline("summarization", model="IlyaGusev/rut5_base_sum_gazeta")
    
    # Разбиваем текст на части
    chunks = chunk_text(text, max_chars=800)
    summaries = []
    
    for i, chunk in enumerate(chunks):
        print(f"Обрабатываю чанк {i+1}/{len(chunks)}...")
        result = summarizer(chunk, max_length=120, min_length=40, do_sample=False)
        summaries.append(result[0]['summary_text'])
    
    final_summary = " ".join(summaries)
    print("\n" + "="*50)
    print("РЕЗУЛЬТАТ СУММАРИЗАЦИИ:")
    print("="*50)
    print(final_summary)
    print("="*50)
    
except Exception as e:
    print(f"Ошибка: {e}")

=== НОВАЯ МОДЕЛЬ 1: IlyaGusev/rut5_base_sum_gazeta ===
Загрузка модели...


Device set to use cpu


Обрабатываю чанк 1/2...


Your max_length is set to 120, but your input_length is only 61. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=30)


Обрабатываю чанк 2/2...

РЕЗУЛЬТАТ СУММАРИЗАЦИИ:
В сети появилось видео, на котором автор рассказывает о красоте математики. Автор объясняет, что математика - это не просто сухие цифры и формулы, а удивительный язык, описывающий вселенную. Математика является красивой наукой, которая помогает нам понимать законы природы и создавать новые технологии. Математика является красивой наукой, которая помогает нам понимать законы природы и создавать новые технологии.


In [11]:
# НОВАЯ МОДЕЛЬ 2: sberbank-ai/ruT5-large
print("=== НОВАЯ МОДЕЛЬ 2: sberbank-ai/ruT5-large ===")
print("Загрузка модели...")

try:
    from transformers import T5ForConditionalGeneration, T5Tokenizer
    import torch
    
    tokenizer = T5Tokenizer.from_pretrained("sberbank-ai/ruT5-large")
    model = T5ForConditionalGeneration.from_pretrained("sberbank-ai/ruT5-large")
    
    # Подготавливаем текст для суммаризации
    input_text = "summarize: " + text
    
    inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True, padding=True)
    
    print("Генерирую суммаризацию...")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=150,
            min_length=50,
            num_beams=4,
            repetition_penalty=2.0,
            length_penalty=1.0,
            early_stopping=True
        )
    
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    print("\n" + "="*50)
    print("РЕЗУЛЬТАТ СУММАРИЗАЦИИ:")
    print("="*50)
    print(summary)
    print("="*50)
    
except Exception as e:
    print(f"Ошибка: {e}")

=== НОВАЯ МОДЕЛЬ 2: sberbank-ai/ruT5-large ===
Загрузка модели...
Генерирую суммаризацию...

РЕЗУЛЬТАТ СУММАРИЗАЦИИ:
summarize: summarize: summarize: summarize: summarize: summarize: summarize:.......


In [12]:
# НОВАЯ МОДЕЛЬ 3: facebook/bart-large-cnn (ИСПРАВЛЕННАЯ ВЕРСИЯ)
print("=== НОВАЯ МОДЕЛЬ 3: facebook/bart-large-cnn ===")
print("Загрузка модели...")

try:
    from transformers import pipeline
    
    summarizer = pipeline("summarization", 
                         model="facebook/bart-large-cnn",
                         tokenizer="facebook/bart-large-cnn")
    
    # Уменьшаем размер чанков для английской модели
    chunks = chunk_text(text, max_chars=512)
    summaries = []
    
    for i, chunk in enumerate(chunks):
        print(f"Обрабатываю чанк {i+1}/{len(chunks)}...")
        try:
            result = summarizer(chunk, 
                              max_length=120, 
                              min_length=30, 
                              do_sample=False,
                              truncation=True)
            summaries.append(result[0]['summary_text'])
        except Exception as e:
            print(f"Ошибка в чанке {i+1}: {e}")
            # Пропускаем проблемный чанк
            continue
    
    if summaries:
        final_summary = " ".join(summaries)
        
        print("\n" + "="*50)
        print("РЕЗУЛЬТАТ СУММАРИЗАЦИИ (английский):")
        print("="*50)
        print(final_summary)
        print("="*50)
    else:
        print("Не удалось сгенерировать суммаризацию")
    
except Exception as e:
    print(f"Ошибка: {e}")

=== НОВАЯ МОДЕЛЬ 3: facebook/bart-large-cnn ===
Загрузка модели...


Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Обрабатываю чанк 1/2...
Обрабатываю чанк 2/2...

РЕЗУЛЬТАТ СУММАРИЗАЦИИ (английский):
На    математика - это  цифры фор щи  язык, опи  ‘физики’,   ‘’Н. ”‘‘Н’’  “”. ””,   ’. ’ ” Каждая  теорема - это  ще    приближающeе нащ  ‘Матектические’: ‘‘’’. ’”. “”’  ’, ””, ‘,’ ”,


In [13]:
# МОДЕЛЬ: 2KKLabs/Lacia_sum_small_v1
print("=== МОДЕЛЬ: 2KKLabs/Lacia_sum_small_v1 ===")
print("Загрузка модели...")

try:
    from transformers import pipeline
    
    summarizer = pipeline(
        "summarization",
        model="2KKLabs/Lacia_sum_small_v1",
        tokenizer="2KKLabs/Lacia_sum_small_v1"
    )
    
    # Разбиваем текст на части
    chunks = chunk_text(text, max_chars=400)
    summaries = []
    
    for i, chunk in enumerate(chunks):
        print(f"Обрабатываю чанк {i+1}/{len(chunks)}...")
        try:
            result = summarizer(
                chunk,
                max_length=120,
                min_length=30,
                do_sample=False,
                truncation=True
            )
            summaries.append(result[0]['summary_text'])
        except Exception as e:
            print(f"Ошибка в чанке {i+1}: {e}")
            continue
    
    if summaries:
        final_summary = " ".join(summaries)
        
        # Если суммаризации слишком длинные, делаем финальное сокращение
        if len(final_summary) > 300:
            try:
                result = summarizer(
                    final_summary,
                    max_length=150,
                    min_length=50,
                    do_sample=False,
                    truncation=True
                )
                final_summary = result[0]['summary_text']
            except:
                pass
        
        print("\n" + "="*50)
        print("РЕЗУЛЬТАТ СУММАРИЗАЦИИ:")
        print("="*50)
        print(final_summary)
        print("="*50)
        
    else:
        print("Не удалось сгенерировать суммаризацию")
    
except Exception as e:
    print(f"Ошибка: {e}")

=== МОДЕЛЬ: 2KKLabs/Lacia_sum_small_v1 ===
Загрузка модели...


Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Your max_length is set to 120, but your input_length is only 92. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)


Обрабатываю чанк 1/3...


Your max_length is set to 120, but your input_length is only 90. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=45)


Обрабатываю чанк 2/3...


Your max_length is set to 120, but your input_length is only 45. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=22)


Обрабатываю чанк 3/3...

РЕЗУЛЬТАТ СУММАРИЗАЦИИ:
**В этом видео рассказывается о красоте математики.** Автор объясняет, что математика - это не просто сухие цифры и формулы, а удивительный язык, описывающий вселенную. Пример числа Пи - бесконечной десятичной дроби, встречающаяся в самых неожиданных местах, таких как геометрия круга и квантовая физика. **Аментальные математические константы**: Математика учит логическому мышлению, помогает находить закономерности и решать сложные проблемы. Каждая доказанная теорема - это маленькое открытие, приближающее нас к пониманию мира. Автор призывает не бояться математики и подчеркивать ее красоту, подчеркивая ее красоту и увлекательность.
